In [114]:
#Imports

%matplotlib inline
import math 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import Perceptron

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


import category_encoders as ce

In [115]:
# Carga de datos.

train =  pd.read_csv('./ComercioExteriorSetDeDatos.csv', encoding='latin-1',dtype={
             "Channel":'category'
            ,"Día":'category'
            ,"Hora":str 
            ,"Periodo del Dia": 'category'
            })

a = train

In [116]:
a.columns

Index(['Channel', 'Día', 'Hora', 'Periodo del Dia', 'Fecha', 'Balenos',
       'Serendia', 'Calpheon ', 'Mediah', 'Valencia', 'Margoria', 'Kamasilvia',
       'Drieghan'],
      dtype='object')

In [117]:
 features = [ 'Hora',
              'Balenos',
              'Serendia',
              'Calpheon ',
              'Mediah',
              'Valencia',
              'Margoria',
              'Kamasilvia',
              'Drieghan',
             'Día_1_oh',
             'Día_2_oh',
             'Día_3_oh',
             'Periodo del Dia_1_oh',
             'Periodo del Dia_2_oh',
             'Periodo del Dia_3_oh',
             'Periodo del Dia_4_oh',
             'Periodo del Dia_5_oh',
             'Periodo del Dia_6_oh'
               ]  # 18
    
features2 = [ 'Hora',
              'Margoria',
             'Día_1_oh',
             'Día_2_oh',
             'Día_3_oh',
             'Periodo del Dia_1_oh',
             'Periodo del Dia_2_oh',
             'Periodo del Dia_3_oh',
             'Periodo del Dia_4_oh',
             'Periodo del Dia_5_oh',
             'Periodo del Dia_6_oh'
               ]  # 11
        
features3 = [ 'Hora',
              'Mediah',
             'Día_1_oh',
             'Día_2_oh',
             'Día_3_oh',
             'Periodo del Dia_1_oh',
             'Periodo del Dia_2_oh',
             'Periodo del Dia_3_oh',
             'Periodo del Dia_4_oh',
             'Periodo del Dia_5_oh',
             'Periodo del Dia_6_oh'
               ] # 11

In [118]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [119]:
def calcularError(model,X_test,X_train ,y_test,y_train):
    #Calculo de Errores contra mi set de test
    pred = model.predict(X_test)

    #Calculo de Errores contra mi set de entrenamiento

    rmsle_test = RMSLE(y_test, pred)
    rmsle_train = RMSLE(y_train, model.predict(X_train) )

    print(f"RMSLE Error (train): {rmsle_train:.5f}")
    print(f"RMSLE Error (Test): {rmsle_test:.5f}")


    mae_test = mean_absolute_error(y_test, pred)
    mae_train = mean_absolute_error(y_train, model.predict(X_train) )

    print(f"MAE Error (train): {mae_train:.5f}")
    print(f"MAE Error (Test): {mae_test:.5f}")

In [120]:
#One hot encoding

one_hot_enc = ce.OneHotEncoder()

one_hot_encoded = one_hot_enc.fit_transform(a[['Día','Periodo del Dia']])

a= a.join(one_hot_encoded.add_suffix("_oh"))

print('One Hot encoding done!')

One Hot encoding done!


In [121]:
a.head()

,Channel,Día,Hora,Periodo del Dia,Fecha,Balenos,Serendia,Calpheon,Mediah,Valencia,Margoria,Kamasilvia,Drieghan,Día_1_oh,Día_2_oh,Día_3_oh,Periodo del Dia_1_oh,Periodo del Dia_2_oh,Periodo del Dia_3_oh,Periodo del Dia_4_oh,Periodo del Dia_5_oh,Periodo del Dia_6_oh
0,Balenos 1,Sábado,23:45:00,(20-24),25/01/20,-1,-1,5,2,-1,4,-1,-1,1,0,0,0,0,0,1,0,0
1,Balenos 2,Sábado,23:48:00,(20-24),25/01/20,1,-1,-1,-1,-1,5,-1,-1,1,0,0,0,0,0,1,0,0
2,Balenos 2,domingo,00:01:00,(0-4 ),26/01/20,-1,4,-1,2,-1,2,-1,-1,0,1,0,1,0,0,0,0,0
3,Calpheon 1,domingo,00:04:00,(0-4 ),26/01/20,-1,-1,5,-1,-1,3,-1,-1,0,1,0,1,0,0,0,0,0
4,Calpheon 1,domingo,07:41:00,(4-8 ),26/01/20,-1,-1,5,-1,-1,3,-1,-1,0,1,0,0,0,0,0,1,0


In [122]:
#transformaciones

a['Fecha'] = pd.to_datetime(a['Fecha'])
a['Hora'] = pd.to_datetime(a['Hora'])

a['Hora'] = a.Hora.dt.hour

In [123]:
#Separo el set 

X = a[features2]

y = a['Channel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=1)

In [126]:

#Modelo

clf = RandomForestClassifier( random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,
                             max_depth = 50,max_features = 11 ,min_samples_split = 2 ,
                             n_estimators = 175 )

#Entrenamiento
model = clf.fit(X_train, y_train)

#Prediccion
print(clf.predict([[ 17,3, 0, 0, 1, 0, 0, 0, 0, 1, 0]]))

['Kamasilvia 1']
